# Higgs uncertainty 

In [1]:
import numpy as np
import pandas as pd
import os
import h5py
import importlib
import tensorflow as tf
import pickle
import yaml
import common.datasets_hephy as datasets_hephy
from data_loader.data_loader_2 import H5DataLoader

2025-04-09 14:36:34.946500: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
model_dir = "./demodir/models"
processes = ['htautau','ztautau','ttbar','diboson']

## Prepare the data

In [2]:
### FIXME: this part should download the data file from Zenodo 
training_data_dir = "/scratch-cbe/users/robert.schoefbeck/Higgs_uncertainty/data/"
selection = "lowMT_VBFJet"

In [4]:
dl = datasets_hephy.get_data_loader(data_directory=training_data_dir, process=None, selection=selection)

In [5]:
### FIXME: maybe we want to add some data plots here

## Train the models

Training the models takes a long time. As a proof of concept, the examples below takes a small fraction of events and train the model for 1 epoch. 

### Inclusive Crosssection

In [7]:
from ML.IC.IC import InclusiveCrosssection

In [8]:
ic_name = f"IC_{selection}"
ic_model_directory = os.path.join( model_dir, "IC" )
os.makedirs(ic_model_directory, exist_ok=True)
filename = os.path.join(ic_model_directory, ic_name)+'.pkl'
print ("Training.")
ic = InclusiveCrosssection()

ic.load_training_data(datasets_hephy, training_data_dir, selection)
#ic.train             (datasets_hephy, mva_selections.selections[args.mvaSelection] if args.mvaSelection is not None else None, small=True)
ic.train             (datasets_hephy, None, small=True)

ic.save(filename)
print ("Written %s"%( filename ))

print(f"Trained IC for this selection: {selection}")
print(ic)

Training.


Computing weight sums:   0%|                                                                        | 0/10 [00:04<?, ?batch/s]

Written ./demodir/models/IC/IC_lowMT_VBFJet.pkl
Trained IC for this selection: lowMT_VBFJet
IC: lowMT_VBFJet                           S/B = 0.003971  yield: htautau:    22.60 ztautau:  4120.57 ttbar:  1539.10 diboson:    32.14
                                                           count: htautau:  1158182 ztautau:   412057 ttbar:   153883 diboson:     3213


### Scalar

In [12]:
from ML.Scaler.Scaler import Scaler
scalar_model_directory = os.path.join(model_dir, "Scaler")
os.makedirs(scalar_model_directory, exist_ok=True)
process_to_train = [None]+processes
for p in process_to_train:
    subdirs = [arg for arg in [p, selection] if arg is not None]
    scaler_name = "Scaler_"+"_".join(subdirs)
    filename = os.path.join(scalar_model_directory, scaler_name) + '.pkl'
    print("Training.")
    scaler = Scaler()

    scaler.load_training_data(training_data_dir=training_data_dir, datasets_hephy=datasets_hephy, selection=selection, process=p)
    scaler.train(small=True)

    scaler.save(filename)
    print(f"Written {filename}")
    

Training.


Computing feature mean/variance:   0%|                                                              | 0/10 [00:05<?, ?batch/s]


Written ./demodir/models/Scaler/Scaler_lowMT_VBFJet.pkl
Training.


Computing feature mean/variance:   0%|                                                              | 0/10 [00:05<?, ?batch/s]


Written ./demodir/models/Scaler/Scaler_htautau_lowMT_VBFJet.pkl
Training.


...

Written ./demodir/models/Scaler/Scaler_ztautau_lowMT_VBFJet.pkl
Training.


Computing feature mean/variance:   0%|                                                              | 0/10 [00:00<?, ?batch/s]


Written ./demodir/models/Scaler/Scaler_ttbar_lowMT_VBFJet.pkl
Training.


Computing feature mean/variance:   0%|                                                              | 0/10 [00:00<?, ?batch/s]

Written ./demodir/models/Scaler/Scaler_diboson_lowMT_VBFJet.pkl


### Inclusive Crosssection Parametrization

In [10]:
from ML.ICP.ICP import InclusiveCrosssectionParametrization

In [11]:
config_name = "ML.configs.icp_quad_tes_jes_met"
# import the training parameters in config
config = importlib.import_module("%s"%( config_name))
for p in processes:
    subdirs = [arg for arg in [p, selection, config_name.split('.')[-1]] if arg is not None]
    icp_name = "ICP_"+"_".join(subdirs)
    icp_model_directory = os.path.join( model_dir, "ICP" )
    os.makedirs(icp_model_directory, exist_ok=True)
    
    filename = os.path.join(icp_model_directory, icp_name)+'.pkl'
    

    print ("Training.")
    icp = InclusiveCrosssectionParametrization( config = config )

    icp.load_training_data(datasets_hephy=datasets_hephy, training_data_dir=training_data_dir, selection=selection, process=p) 
    icp.train             (small=True, train_ratio = True, selection=None)

    icp.save(filename)
    print ("Written %s"%( filename ))
    
    print (f"Trained ICP with config {config_name} in selection {selection}")
    prefix = "ICP: "+selection
    print (prefix.ljust(50)+icp.__str__())

Training.
ICP training data: Base point nu = (0.0, 0.0, 0.0), alpha = (1.0, 1.0, 0.0), file = /scratch-cbe/users/robert.schoefbeck/Higgs_uncertainty/data/lowMT_VBFJet/htautau.h5
ICP training data: Base point nu = (-3.0, 0.0, 0.0), alpha = (0.97, 1.0, 0.0), file = /scratch-cbe/users/robert.schoefbeck/Higgs_uncertainty/data/lowMT_VBFJet/htautau_tes_0p97.h5
ICP training data: Base point nu = (-2.0, 0.0, 0.0), alpha = (0.98, 1.0, 0.0), file = /scratch-cbe/users/robert.schoefbeck/Higgs_uncertainty/data/lowMT_VBFJet/htautau_tes_0p98.h5
ICP training data: Base point nu = (-1.0, -1.0, 0.0), alpha = (0.99, 0.99, 0.0), file = /scratch-cbe/users/robert.schoefbeck/Higgs_uncertainty/data/lowMT_VBFJet/htautau_tes_0p99_jes_0p99.h5
ICP training data: Base point nu = (-1.0, -1.0, 1.0), alpha = (0.99, 0.99, 1.0), file = /scratch-cbe/users/robert.schoefbeck/Higgs_uncertainty/data/lowMT_VBFJet/htautau_tes_0p99_jes_0p99_met_1.h5
ICP training data: Base point nu = (-1.0, -1.0, 2.0), alpha = (0.99, 0.99, 2.0

Computing weight sum:   0%|                                                                         | 0/10 [00:05<?, ?batch/s]


Written ./demodir/models/ICP/ICP_htautau_lowMT_VBFJet_icp_quad_tes_jes_met.pkl
Trained ICP with config ML.configs.icp_quad_tes_jes_met in selection lowMT_VBFJet
ICP: lowMT_VBFJet                                 +6.0e-03*nu_tes +1.3e-02*nu_jes -3.4e-05*nu_met -9.9e-05*nu_tes*nu_tes -1.6e-04*nu_jes*nu_jes -9.2e-05*nu_met*nu_met +4.6e-05*nu_tes*nu_jes -4.5e-06*nu_tes*nu_met +5.2e-06*nu_jes*nu_met
Training.
ICP training data: Base point nu = (0.0, 0.0, 0.0), alpha = (1.0, 1.0, 0.0), file = /scratch-cbe/users/robert.schoefbeck/Higgs_uncertainty/data/lowMT_VBFJet/ztautau.h5
ICP training data: Base point nu = (-3.0, 0.0, 0.0), alpha = (0.97, 1.0, 0.0), file = /scratch-cbe/users/robert.schoefbeck/Higgs_uncertainty/data/lowMT_VBFJet/ztautau_tes_0p97.h5
ICP training data: Base point nu = (-2.0, 0.0, 0.0), alpha = (0.98, 1.0, 0.0), file = /scratch-cbe/users/robert.schoefbeck/Higgs_uncertainty/data/lowMT_VBFJet/ztautau_tes_0p98.h5
ICP training data: Base point nu = (-1.0, -1.0, 0.0), alpha = (0.99

### Multiclassifier

In [13]:
from ML.TFMC.TFMC import TFMC

Num GPUs Available:  0


In [14]:
config_name = "ML.configs.tfmc_scan_do_1"
# import the training parameters in config
config = importlib.import_module("%s"%( config_name))

# Where to store the training
TFMC_model_directory = os.path.join(model_dir, "TFMC", selection, config_name.split('.')[-1])
os.makedirs(TFMC_model_directory, exist_ok=True)

In [15]:
if config.use_ic:
    from ML.IC.IC import InclusiveCrosssection
    ic = InclusiveCrosssection.load(os.path.join(model_dir, "IC", "IC_"+selection+'.pkl'))
    config.weight_sums = ic.weight_sums
    print("We use this IC:")
    print(ic)
    
# Do we use a Scaler?
if config.use_scaler:
    from ML.Scaler.Scaler import Scaler 
    scaler = Scaler.load(os.path.join(model_dir, "Scaler", "Scaler_"+selection+'.pkl'))
    config.feature_means     = scaler.feature_means
    config.feature_variances = scaler.feature_variances

    print("We use this scaler:")
    print(scaler)

tfmc = TFMC(config)

tfmc.load_training_data(datasets_hephy, training_data_dir, selection, n_split=100)

max_batch = 1 # FIXME: -1 for full training

# Determine the starting epoch
starting_epoch = 0

# Training Loop
for epoch in range(0, config.n_epochs):

    # Manually evaluate and update the learning rate
    if hasattr(tfmc, 'lr_schedule'):  # Ensure the schedule exists
        new_lr = tfmc.lr_schedule(epoch)
        tfmc.optimizer.learning_rate.assign(new_lr)  # Update the optimizer's learning rate

    # Print the current learning rate
    current_lr = tf.keras.backend.get_value(tfmc.optimizer.learning_rate)  # Direct access
    print(f"Epoch {epoch}/{config.n_epochs} - Learning rate: {current_lr:.6f}")

    true_histograms, pred_histograms = tfmc.train_one_epoch(max_batch=max_batch, accumulate_histograms=(epoch%1==0))
    tfmc.save(TFMC_model_directory, epoch)  # Save model and config after each epoch

    #if true_histograms is not None and pred_histograms is not None:
    #    # Plot convergence
    #    tfmc.plot_convergence_root(
    #        true_histograms,
    #        pred_histograms,
    #        epoch,
    #        plot_directory,
    #        data_structure.feature_names, 
    #    )

    break

print(f"TFMC models saved in {TFMC_model_directory}")

We use this IC:
IC: lowMT_VBFJet                           S/B = 0.003971  yield: htautau:    22.60 ztautau:  4120.57 ttbar:  1539.10 diboson:    32.14
                                                           count: htautau:  1158182 ztautau:   412057 ttbar:   153883 diboson:     3213
We use this scaler:
Scaler: selection lowMT_VBFJet process (not set)
PRI_lep_pt: mean=46.312, variance=850.590
PRI_lep_eta: mean=-0.000, variance=1.328
PRI_lep_phi: mean=-0.002, variance=3.288
PRI_had_pt: mean=61.323, variance=1443.225
PRI_had_eta: mean=0.001, variance=1.393
PRI_had_phi: mean=0.003, variance=3.287
PRI_jet_leading_pt: mean=119.773, variance=5152.242
PRI_jet_leading_eta: mean=-0.001, variance=3.265
PRI_jet_leading_phi: mean=-0.002, variance=3.275
PRI_jet_subleading_pt: mean=61.797, variance=1126.916
PRI_jet_subleading_eta: mean=0.002, variance=4.463
PRI_jet_subleading_phi: mean=-0.001, variance=3.246
PRI_n_jets: mean=2.895, variance=1.162
PRI_jet_all_pt: mean=216.515, variance=14143.631
P

2025-04-08 22:14:41.869556: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 0/300 - Learning rate: 0.010000


Processing Batches:   0%|                                                                             | 0/100 [00:02<?, ?it/s]

TFMC models saved in ./demodir/models/TFMC/lowMT_VBFJet/tfmc_scan_do_1


### Calibrator

In [10]:
### FIXME: not sure how is the calibrator trained

### PNN

In [16]:
from ML.PNN.PNN import PNN

In [17]:
config_name = "ML.configs.pnn_quad_tes_jes_met"
# import the training parameters in config
config = importlib.import_module("%s"%( config_name))

for p in processes:
    subdirs= [arg for arg in [p, selection] if arg is not None]

    # Do we use ICP?
    if config.icp is not None:
        from ML.ICP.ICP import InclusiveCrosssectionParametrization
        icp_name = "ICP_"+"_".join(subdirs)+"_"+config.icp+".pkl"
        icp = InclusiveCrosssectionParametrization.load(os.path.join(model_dir, "ICP", icp_name))
        config.icp_predictor = icp.get_predictor()
        print("We use this ICP:",icp_name)
        print(icp)
    
    # Do we use a Scaler?
    if config.use_scaler:
        from ML.Scaler.Scaler import Scaler
        scaler_name = "Scaler_"+"_".join(subdirs)+'.pkl'
        scaler = Scaler.load(os.path.join(model_dir, "Scaler", scaler_name))
        config.feature_means     = scaler.feature_means
        config.feature_variances = scaler.feature_variances
    
        print("We use this scaler:", scaler_name)
        print(scaler)
    
    # Where to store the training
    pnn_model_directory = os.path.join(model_dir, "PNN", *subdirs,  config_name)
    os.makedirs(pnn_model_directory, exist_ok=True)

# Initialize model
pnn = PNN(config)

# Initialize for training
pnn.load_training_data(datasets_hephy=datasets_hephy, training_data_dir=training_data_dir, process=p, selection=selection, n_split=100)

max_batch = 1 #if args.small else -1

# Training Loop
for epoch in range(0, config.n_epochs):

    # Manually evaluate and update the learning rate
    if hasattr(pnn, 'lr_schedule'):  # Ensure the schedule exists
        new_lr = pnn.lr_schedule(epoch)
        pnn.optimizer.learning_rate.assign(new_lr)  # Update the optimizer's learning rate
  
    # Print the current learning rate
    current_lr = tf.keras.backend.get_value(pnn.optimizer.learning_rate)  # Direct access
    print(f"Epoch {epoch}/{config.n_epochs} - Learning rate: {current_lr:.6f}")

    true_histograms, pred_histograms = pnn.train_one_epoch(max_batch=max_batch, accumulate_histograms=(epoch%1==0), rebin=1)
    pnn.save(pnn_model_directory, epoch)  # Save model and config after each epoch

    break


print(f"PNN models saved in {pnn_model_directory}")

We use this ICP: ICP_htautau_lowMT_VBFJet_icp_quad_tes_jes_met.pkl
+6.0e-03*nu_tes +1.3e-02*nu_jes -3.4e-05*nu_met -9.9e-05*nu_tes*nu_tes -1.6e-04*nu_jes*nu_jes -9.2e-05*nu_met*nu_met +4.6e-05*nu_tes*nu_jes -4.5e-06*nu_tes*nu_met +5.2e-06*nu_jes*nu_met
We use this scaler: Scaler_htautau_lowMT_VBFJet.pkl
Scaler: selection lowMT_VBFJet process (htautau)
PRI_lep_pt: mean=47.784, variance=934.071
PRI_lep_eta: mean=0.000, variance=1.262
PRI_lep_phi: mean=-0.001, variance=3.288
PRI_had_pt: mean=64.702, variance=1609.351
PRI_had_eta: mean=0.001, variance=1.324
PRI_had_phi: mean=0.003, variance=3.285
PRI_jet_leading_pt: mean=127.257, variance=5997.741
PRI_jet_leading_eta: mean=-0.001, variance=3.695
PRI_jet_leading_phi: mean=-0.001, variance=3.269
PRI_jet_subleading_pt: mean=63.424, variance=1245.298
PRI_jet_subleading_eta: mean=0.000, variance=4.870
PRI_jet_subleading_phi: mean=-0.002, variance=3.229
PRI_n_jets: mean=2.893, variance=1.149
PRI_jet_all_pt: mean=225.778, variance=15460.502
PRI_m

Processing Batches: 0it [00:00, ?it/s]

PNN models saved in ./demodir/models/PNN/diboson/lowMT_VBFJet/ML.configs.pnn_quad_tes_jes_met


## Predict

For the predictions performed below, the fully trained model provided in `./models` are used.

In [5]:
from Workflow.Inference import Inference
from common.likelihoodFit import likelihoodFit
from common.intervalFinder import intervalFinder

Num GPUs Available:  0


### Load test dataset and config

A config file `./configs/config_submission.yaml` is used to control the path of the ML models. Please find the detailed information in the README of the GitHub repository.

In [6]:
def load_h5_to_test_set(h5_file_path):
    with h5py.File(h5_file_path, "r") as hf:
        full_data = hf["data"][:]  # (N, 30)
    test_data = pd.DataFrame(full_data[:, :28])  # (N, 28)
    test_weights = full_data[:, 28]  # (N,)
    test_set = {
        "data": test_data,
        "weights": test_weights
    }
    return test_set

def loadConfig(config_path):
    if config_path.endswith(".pkl"):
        use_yaml = False
    else:
        try:
            import yaml
            use_yaml = True
        except:
            import pickle
            use_yaml = False
            config_path = config_path.replace(".yaml", ".pkl")

    assert os.path.exists(config_path), "Config does not exist: {}".format(config_path)

    if use_yaml:
        with open(config_path) as f:
            cfg = yaml.safe_load(f)
    else:
        with open(config_path) as f:
            cfg = pickle.load(f, 'rb')

    for task in cfg["Tasks"]:
        for selection in cfg["Selections"]:
            for item in ["calibration", "icp_file", "model_path"]:
                if item in cfg[task][selection]:
                    cfg[task][selection][item] = cfg[task][selection][item]

    if "Poisson" in cfg:
        for sel in cfg["Poisson"].keys():
            if 'model_path' in cfg["Poisson"][sel]:
                cfg["Poisson"][sel]["model_path"] = cfg["Poisson"][sel]["model_path"]
            cfg["Poisson"][sel]["IC"] = cfg["Poisson"][sel]["IC"]
            for process in cfg["Poisson"][sel]["ICP"].keys():
                cfg["Poisson"][sel]["ICP"][process] = cfg["Poisson"][sel]["ICP"][process]
    
    cfg['tmp_path'] = f"data/tmp_data"

    return cfg

In [7]:
test_dataset_path = "/eos/vbc/group/mlearning/data/Higgs_uncertainty/input_data/split_train_dataset/processed_data/pseudo_experiments_with_true_labels_mu_2/set_2.0_pseudo_exp_10.h5"
test_dataset = load_h5_to_test_set(test_dataset_path)

predict_config = "configs/config_submission.yaml"

### Pre-save information for training dataset

The prediction requires information from the training dataset for the inclusive cross section term. To reduce the runtime of the prediction steps, we pre-save the ML output for the training data, and save the cubic spline interpolation (CSI) for inclusive cross section and poisson terms. 

The pre-saved files are all data with the fully trained ML models are availible in `data/tmp_data`. Those files will be used later in the prediction.

In [11]:
cfg = loadConfig(predict_config)
cfg['training_data_dir'] = training_data_dir
infer = Inference(cfg, small=True, overwrite=True, toy_origin="config", toy_path=None, toy_from_memory=None)
infer.cfg["CSI"]["save"] = False
cfg['tmp_path'] = "demodir/tmp_data"
infer.save(restrict_csis = [])
for p in processes:
    infer = Inference(cfg, small=True, overwrite=True, toy_origin="config", toy_path=None, toy_from_memory=None)
    infer.cfg["CSI"]["save"] = True
    cfg['tmp_path'] = "demodir/tmp_data"
    infer.save(restrict_csis = [p])

Processing batches:   1%|          | 1/100 [00:04<08:05,  4.90s/it]
Warning! Temporary file demodir/tmp_data/nominal_lowMT_VBFJet.h5 exists. It will be overwritten.
Processing batches:   0%|          | 0/100 [00:02<?, ?it/s]
Warning! Temporary file demodir/tmp_data/TrainingData_lowMT_VBFJet.h5 exists. It will be overwritten.
CSI lowMT_VBFJet htautau: 100%|██████████| 2/2 [00:22<00:00, 11.29s/it]
Warning! Temporary file demodir/tmp_data/nominal_lowMT_noVBFJet_ptH100.h5 exists. It will be overwritten.
Processing batches:   0%|          | 0/100 [00:00<?, ?it/s]
Warning! Temporary file demodir/tmp_data/TrainingData_lowMT_noVBFJet_ptH100.h5 exists. It will be overwritten.
CSI lowMT_noVBFJet_ptH100 htautau: 100%|██████████| 1/1 [00:06<00:00,  6.06s/it]
Warning! Temporary file demodir/tmp_data/nominal_highMT_VBFJet.h5 exists. It will be overwritten.
Processing batches:   0%|          | 0/100 [00:00<?, ?it/s]
Warning! Temporary file demodir/tmp_data/TrainingData_highMT_VBFJet.h5 exists. It wil

### Construct the likelyhood function and extract the interval

In [14]:

offset = 0.0
inflate = 1.045

infer = Inference(cfg=cfg, small=False, overwrite=False, toy_origin="memory", toy_path=None, toy_from_memory=None)
infer.ignore_loading_check()

# Initialize inference object
infer.setToyFromMemory(test_dataset)
infer._dcr_cache = {}

# Define likelihood function
likelihood_function = lambda mu, nu_bkg, nu_tt, nu_diboson, nu_tes, nu_jes, nu_met: \
    infer.predict(mu=mu, nu_bkg=nu_bkg, nu_tt=nu_tt, nu_diboson=nu_diboson, \
    nu_tes=nu_tes, nu_jes=nu_jes, nu_met=nu_met, \
    asimov_mu=None, \
    asimov_nu_bkg=None, \
    asimov_nu_tt=None, \
    asimov_nu_diboson=None)

# Perform global fit
fit = likelihoodFit(likelihood_function)
fit.parameterBoundaries["mu"] = (0, None)
q_mle, parameters_mle, cov, limits = fit.fit(start_mu=1.0)

mu_mle = parameters_mle["mu"]
delta_mu = np.sqrt(cov["mu", "mu"])
p16 = mu_mle - delta_mu
p84 = mu_mle + delta_mu

# Now do NON-PROFILED scan
Npoints = 21
mumin = min(mu_mle - 3*delta_mu, mu_mle-0.5) # if delta_mu is too small, scan from mu-0.5 to mu+0.5
mumax = max(mu_mle + 3*delta_mu, mu_mle+0.5) # if delta_mu is too small, scan from mu-0.5 to mu+0.5

# Now go to MLE point and only evaluate mu
deltaQ = []
muPoints = [mumin+i*(mumax-mumin)/Npoints for i in range(Npoints)]

for i, mu in enumerate(muPoints):
    q = likelihood_function(mu=mu, nu_bkg=parameters_mle["nu_bkg"], nu_tt=parameters_mle["nu_tt"], nu_diboson=parameters_mle["nu_diboson"], nu_tes=parameters_mle["nu_tes"], nu_jes=parameters_mle["nu_jes"], nu_met=parameters_mle["nu_met"])
    deltaQ.append(q-q_mle)

# Interval finder interpolates and returns crossing points
# if a boundary is below best fit mu, it is the lower boundary, if above it is the upper

intFinder = intervalFinder(muPoints, deltaQ, 1.0)
boundaries = intFinder.getInterval()
for b in boundaries:
    if b < mu_mle:
        p16 = b
    if b > mu_mle:
        p84 = b

# inflate and offset

p16 = mu_mle - inflate*(mu_mle-p16) + offset
p84 = mu_mle + inflate*(p84-mu_mle) + offset
mu_mle = mu_mle + offset

# Check mu boundaries
if p16 < 0.1:
    p16 = 0.09
if p84 > 3.0:
    p84 = 3.01

delta_mu = (p84-p16)/2

results = {
            "mu_hat": mu_mle,
            "delta_mu_hat": delta_mu,
            "p16": p16,
            "p84": p84,
            "nu_bkg": parameters_mle["nu_bkg"],
            "nu_tt":  parameters_mle["nu_tt"],
            "nu_diboson": parameters_mle["nu_diboson"],
            "nu_tes": parameters_mle["nu_tes"],
            "nu_jes": parameters_mle["nu_jes"],
            "nu_met": parameters_mle["nu_met"],
        }

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = -5.977e+04                 │              Nfcn = 415              │
│ EDM = 2.73e-06 (Goal: 0.0001)    │            time = 6.4 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│     SOME parameters at limit     │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬────────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬

### Results

In [15]:
print(results)

{'mu_hat': 361.1596758617736, 'delta_mu_hat': -178.28152296167193, 'p16': 359.57304592334384, 'p84': 3.01, 'nu_bkg': -5.711534845999731, 'nu_tt': 9.999999999482512, 'nu_diboson': -2.84210418098054, 'nu_tes': -9.99999999972082, 'nu_jes': 2.2470104072546913, 'nu_met': 4.99999999926586}
